# Generate automatic documentation for individual project

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
PROJECT_KEY = 'TIM_HEALTH_DATA'
DATASET_ID = 'Apple_Health_daily_summaries'

In [0]:
client = dataiku.api_client()
project = client.get_project(PROJECT_KEY)
dataset_handle = project.get_dataset(DATASET_ID)

In [0]:
dataset_handle